# open problems (task batch correction / label proj)


In [5]:
import pandas as pd
import requests
import json
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection
import scanpy as sc
from scprint import scPrint
from scdataloader import Preprocessor
from scprint.tasks import Embedder

→ connected lamindb: jkobject/scprint2


/pasteur/appa/homes/jkalfon/simpler_flash/src/simpler_flash/layer_norm.py:1044: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/pasteur/appa/homes/jkalfon/simpler_flash/src/simpler_flash/layer_norm.py:1107: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [2]:
! uv pip list | grep scib #same version as OP

Using Python 3.11.11 environment at: /pasteur/appa/homes/jkalfon/scPRINT/.venv
scib                       1.1.7
scib-metrics               0.5.6


In [6]:
LOC = "/pasteur/appa/scratch/jkalfon/data/spcrint_data/"

In [ ]:
url = "https://raw.githubusercontent.com/openproblems-bio/website/main/results/batch_integration/data/results.json"
response = requests.get(url)

with open("results_batch.json", "w") as f:
    f.write(response.text)

url = "https://raw.githubusercontent.com/openproblems-bio/website/main/results/label_projection/data/results.json"
response = requests.get(url)

with open("results_label.json", "w") as f:
    f.write(response.text)

print("File downloaded successfully!")

File downloaded successfully!


In [ ]:
print("File downloaded successfully!")
with open("results_batch.json", "r") as f:
    data_batch = json.load(f)

File downloaded successfully!


In [22]:
res = {}
for dataset in data_batch:
    dataset_id = dataset["dataset_id"]
    if dataset_id not in res:
        res[dataset_id] = {}
    res[dataset_id].update({dataset["method_id"]: dataset["metric_values"]})

In [18]:
with open("results_label.json", "r") as f:
    data_label = json.load(f)

In [23]:
res_label = {}
for dataset in data_label:
    dataset_id = dataset["dataset_id"]
    if dataset_id not in res_label:
        res_label[dataset_id] = {}
    res_label[dataset_id].update({dataset["method_id"]: dataset["metric_values"]})

In [24]:
res_label.keys()

dict_keys(['cellxgene_census/dkd', 'cellxgene_census/gtex_v9', 'cellxgene_census/hypomap', 'cellxgene_census/immune_cell_atlas', 'cellxgene_census/mouse_pancreas_atlas', 'cellxgene_census/tabula_sapiens', None])

In [25]:
pd.DataFrame(res_label["cellxgene_census/gtex_v9"])

,knn,logistic_regression,majority_vote,mlp,naive_bayes,random_labels,scanvi,scanvi_scarches,scgpt_zeroshot,scimilarity,scimilarity_knn,seurat_transferdata,singler,true_labels,uce,xgboost,geneformer,scgpt_finetuned,scprint
accuracy,0.8523,0.8829,0.0799,0.7784,0.7599,0.0324,0.8899,0.8797,0.6234,0.6665,0.8253,0.8410,0.7903,1,0.0054,0.8328,NA,NA,NA
f1_macro,0.3913,0.4437,0.0082,0.2948,0.3363,0.0071,0.4083,0.4555,0.2534,0.3254,0.4563,0.4186,0.3753,1,0.0037,0.3251,NA,NA,NA
f1_micro,0.8523,0.8829,0.0799,0.7784,0.7599,0.0324,0.8899,0.8797,0.6234,0.6665,0.8253,0.8410,0.7903,1,0.0054,0.8328,NA,NA,NA
f1_weighted,0.8582,0.8851,0.0118,0.7896,0.7937,0.0397,0.9019,0.8908,0.6375,0.6504,0.8386,0.8457,0.8230,1,0.0082,0.8408,NA,NA,NA


In [ ]:
pd.DataFrame(res["cellxgene_census/gtex_v9"])

In [ ]:
model_checkpoint_file = "../models/ogvvg2z7-v1.ckpt"

In [ ]:
model = scPrint.load_from_checkpoint(
    model_checkpoint_file, precpt_gene_emb=None, gene_pos_file=None
)
model = model.to("cuda")

In [3]:
datasets = {
    "cellxgene_census/dkd": "https://datasets.cellxgene.cziscience.com/46d8d92b-32e0-4ca5-9907-4dbf519c7fc3.h5ad",  # 0.3
    "cellxgene_census/gtex_v9": "https://datasets.cellxgene.cziscience.com/002308e1-0121-4aa1-b8f2-9d034cf44b0f.h5ad",  # 1gb
    "cellxgene_census/hypomap": "https://datasets.cellxgene.cziscience.com/d3be7423-d664-4913-89a9-a506cae4c28f.h5ad",  # 4gb
    "cellxgene_census/mouse_pancreas_atlas": "https://datasets.cellxgene.cziscience.com/49243c50-bf0c-4b10-87f8-55ec9f455399.h5ad",  # 4gb
    "cellxgene_census/immune_cell_atlas": "https://datasets.cellxgene.cziscience.com/78819b62-0699-4672-8dc8-d9317b04d255.h5ad",  # 3gb
    # 'cellxgene_census/tabula_sapiens': 'https://datasets.cellxgene.cziscience.com/5a495302-b7cd-4bf9-853e-95627b00bb03.h5ad' # 42gb
}

LOC = "/pasteur/appa/scratch/jkalfon/data/spcrint_data/temp/"  # "../../data/temp/"  #

In [ ]:
metrics = {}
for name, url in list(datasets.items())[:]:
    print("doing ", name)
    adata = sc.read(LOC + name + ".h5ad", backup_url=url)
    is_symbol = False
    preprocessor = Preprocessor(
        force_preprocess=True,
        skip_validate=True,
        # drop_non_primary=False,
        is_symbol=is_symbol,
        do_postp=model.expr_emb_style == "metacell",
    )
    print("")
    adata = preprocessor(adata)
    if model.expr_emb_style == "metacell":
        sc.pp.neighbors(adata, use_rep="X_pca")

    embed = Embedder(
        how="random expr",
        max_len=4000,
        num_workers=8,
        pred_embedding=["cell_type_ontology_term_id"],
        doplot=False,
    )
    _, metrics[name] = embed(model, adata)
    print(metrics[name])
    bm = Benchmarker(
        adata,
        batch_key="assay_ontology_term_id",  # "batch",  # batch, tech, assay_ontology_term_id, donor_id
        label_key="cell_type",  # celltype
        embedding_obsm_keys=["scprint_emb"],
        bio_conservation_metrics=BioConservation(),
        batch_correction_metrics=None,
        n_jobs=10,
    )
    bm.benchmark()
    print(bm.get_results())
    metrics[name].update({'batch_corr': bm.get_results()})
    bm.plot_results_table(min_max_scale=False)

doing  cellxgene_census/dkd


  0%|          | 0.00/384M [00:00<?, ?B/s]

In [ ]:
metrics